In [2]:
import requests
from bs4 import BeautifulSoup
import re

from config import gaishi, minshuu, vorkers
from selenium import webdriver
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

import json
from pprint import pprint

%matplotlib inline

In [8]:
company_dics = {
    "三菱商事": {
        "name": "mitsubishi_shouji",
        "gaishi": "37",
        "vorkers": "a0910000000Frj0"
    },
}

In [ ]:
# 外資就活ドットコム
SR_SAVE_FORMAT = "./data/selection_reports/%s.json"
gaishi_driver = GaishiDriver()
for company, company_dic in company_dics.items():
    print(company)
    name = company_dic["name"]
    SR_SAVE_PATH = SR_SAVE_FORMAT % name
    gaishi_driver.access_sr_list_page(company_number=company_dic["gaishi"])
    sr_urls = gaishi_driver.get_sr_urls()
    srs = gaishi_driver.get_each_sr(sr_urls)
    with open(SR_SAVE_PATH, "w") as f:
        json.dump(srs, f)


In [ ]:
# Vorkers
RV_SAVE_FORMAT = "./data/reviews/%s.json"
vorkers_driver = VorkersDriver()
for company, company_dic in company_dics.items():
    print(company)
    name = company_dic["name"]
    RV_SAVE_PATH = RV_SAVE_FORMAT % name
    vorkers_driver.access_top_page(company_id=company_dic["vorkers"])
    vorkers_driver.get_score()
    vorkers_driver.access_review_page(company_id=company_dic["vorkers"])
    dic = vorkers_driver.get_reviews()
    with open(RV_SAVE_PATH, "w") as f:
        json.dump(dic, f)

In [18]:
import json
with open("./data/reviews/mitsubishi_shouji.json", "r") as f:
    reviews = json.load(f)["reviews"]
pprint(reviews)

['日本の事業会社固有の形式・段階を経た物事の進めかたなど堅い面は否定しきれないが、日本を代表する企業としては必要な面であるのかもしれない。（必要悪といえるかもしれない。）トレーディングから事業投資･事業経営に重心を変えてきた過程の10-20年において過去の良き奔放さなどは薄らいできているように感じる。日本の商社の強み･ユニークさは若い年代からトレーディング売買(相場を追いかける･相場に追いかけまくられる売買、開発を伴う売買など）を通して実際のモノの動きに関わり経験を積んだ上で、そこから先の事業投資経営に自己で繋げていくことにあったと自身の経験を振り返るが、最近は若手にトレーディングを経験する機会がないことが人材育成にどう影響していくのか考えさせられる。\n'
 'また事業投資･事業経営のマネージメント(管理）･統治をいまだにトレーディングのリスク管理の視点から見てしまっている点も散見される。当たり前ながら製造・技術開発の経営資源がどうしても抜け落ちてしまうこと、事業経営にてその部分を如何に補うかが課題であろうが、それをも取り込んでいこうとする姿勢は強い企業たる所以に思える。\n'
 '人材は極めて優秀、日本的な人事性向(派閥・力関係など）もなくはないが他企業と比較するとそれが重きでないように見える。',
 'コンプライアンスに対する意識が非常に高く、きれいごとがまかり通るため安心して仕事ができる。\n'
 '本当に実力のある人間が評価されるとは限らず、社内政治だけで泳いでいる輩もかなりいる。\n'
 '30台半ばから表街道を行くグループと外れたグループが分かれ始め、どの年次でどういう部署・出向先に異動になったで察することができるが、金銭的な待遇という面ではそれほど差は出ない。金銭的な報酬は役員にでもならない限り、出世したかしてないかではなく、どの国にどれくらい駐在したかが大きく影響する。',
 '会社としてかなり完成されており、経営陣の入れ替えも大きなサプライズなく周期的に行われていて、非常に安定している。\n'
 '昔の投資（資源や自動車での大博打と成功）は面白かったという話を先輩からよく聞いたが、最近は投資も控えめな空気があり、部署間の気の遣い合いと牽制の比重が大きくなっているように感じているが、このまま社内政治と根回し偏重のままでは、伊藤忠にトップを獲られる

## 外資就活ドットコム

In [3]:
sr_list_page = "https://gaishishukatsu.com/company/%s/study"

class GaishiDriver(object):
    
    def __init__(self,):
        self.driver = webdriver.PhantomJS(service_log_path=os.path.devnull)
        self.es_list_page = None
        self.login()
        
    def login(self,):
        self.driver.get("https://gaishishukatsu.com/login")
        self.driver.implicitly_wait(1)
        username_field = self.driver.find_element_by_name("data[GsUser][email]")
        password_field = self.driver.find_element_by_name("data[GsUser][password]")
        username_field.send_keys(gaishi["email"])
        self.driver.implicitly_wait(1)
        password_field.send_keys(gaishi["password"])
        self.driver.implicitly_wait(1)
        self.driver.find_element_by_css_selector("#GsUserLoginForm > div._mt-md.content-center-button > p:nth-child(1) > button").click()
        
    def access_sr_list_page(self, company_number):
        self.driver.implicitly_wait(1)
        self.driver.get(sr_list_page % company_number)
        source = self.driver.page_source.encode('utf-8')
        self.sr_list_page = BeautifulSoup(source, "lxml")
        
    def get_sr_urls(self,):
        sr_urls = []
        es_list = self.sr_list_page.find_all("a", class_="_panel-content report-list")
        for es in es_list:
            url = es["ng-href"]
            if str(url).startswith("https://gaishishukatsu.com/selection_reports/"):
                sr_urls.append(url)
        return sr_urls
    
    def get_each_sr(self, sr_urls):
        srs = {}
        for sr_url in sr_urls:
            self.driver.implicitly_wait(1)
            self.driver.get(sr_url)
            source = self.driver.page_source.encode('utf-8')
            sr_page = BeautifulSoup(source, "lxml")
            try:
                importance = sr_page.find("td", class_="impression_importance").text
            except AttributeError:
                importance = ""
            try:
                advice = sr_page.find("td", class_="impression_advice").text
            except:
                advice = ""
            srs[sr_url] = {"importance": importance, "advice": advice}        
        return srs


## VORKERS

In [11]:
top_page = "https://www.vorkers.com/company.php?m_id=%s"
review_page = "https://www.vorkers.com/company_answer.php?m_id=%s&q_no=1"

class VorkersDriver(object):
    
    def __init__(self,):
        self.driver = webdriver.PhantomJS(service_log_path=os.path.devnull)
        self.top_page = None
        self.review_page = None
        self.dic = {}
        self.login()
        
    def login(self,):
        self.driver.get("https://www.vorkers.com/login.php")
        self.driver.implicitly_wait(1)
        username_field = self.driver.find_element_by_name("_username")
        password_field = self.driver.find_element_by_name("_password")
        username_field.send_keys(vorkers["email"])
        self.driver.implicitly_wait(1)
        password_field.send_keys(vorkers["password"])
        self.driver.implicitly_wait(1)
        self.driver.find_element_by_css_selector("#log_in").click()
        print("login")
        
    def access_top_page(self, company_id):
        self.driver.implicitly_wait(1)
        self.driver.get(top_page % company_id)
        source = self.driver.page_source.encode('utf-8')
        self.top_page = BeautifulSoup(source, "lxml")
        print("top page")

    def get_score(self,):
        # レーダーチャートのスコア
        radar_elems = self.top_page.find("ul", class_="scoreList-8").find_all("li")
        self.dic["radar"] = {}
        radar_dic = self.dic["radar"]
        for radar_elem in radar_elems:
                key = radar_elem.find("dt").text
                score = radar_elem.find("dd").text
                radar_dic[key] = score
        # 総合評価
        self.dic["総合評価"] = self.top_page.find("span", property="v:average").text
        # 残業時間(月間)と有給休暇消化率
        zangyou, yuukyuu = self.top_page.find_all("dd", class_="d-ib t-r w-85")
        self.dic["残業時間(月間)"] = zangyou.text
        self.dic["有給休暇消化率"] = yuukyuu.text
        print("got score")
        
    def access_review_page(self, company_id):
        self.driver.implicitly_wait(1)
        self.driver.get(review_page % company_id)
        source = self.driver.page_source.encode('utf-8')
        self.review_page = BeautifulSoup(source, "lxml")
        print("review page 1")

    def get_reviews(self,):
        reviews = self.review_page.find_all("dd", class_="article_answer")
        review_arr = [review.text.strip("\n").strip() for review in reviews]
        next_pages = self.review_page.find_all("a", class_="paging_link-item")
        if len(next_pages):
            for i, next_page in enumerate(next_pages):
                print("review page ", i+2)
                self.driver.implicitly_wait(3)
                url = next_page.get("href")
                self.driver.get(url)
                source = self.driver.page_source.encode('utf-8')
                self.review_page = BeautifulSoup(source, "lxml")
                reviews = self.review_page.find_all("dd", class_="article_answer")
                review_arr += [review.text.strip("\n").strip() for review in reviews]
        self.dic["reviews"] = review_arr
        return self.dic